# Instalación e importación de dependencias

In [ ]:
!pip install gradio
!sudo install ffmpeg
!pip install streamlit
!pip install streamlit-lottie
!pip install -U openai-whisper
!pip install scikit-learn
!pip install resampy
!pip install scipy
!pip install oracledb
!pip install pyAudioAnalysis
#!pip install hmmlearn
#!pip install eyed3
!pip install librosa
!pip install pydub
!pip install simpleaudio
!pip install pyannote.audio
!pip install tabulate

%pip install -U numpy==1.23.5
%pip install -U pandas==2.0.0
%pip install IPython
%pip install -U torch==2.0.0+cu118
%pip install torchaudio tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
import os
import glob
import shutil
import joblib
import evaluate
import oracledb
import subprocess
import numpy as np
import gradio as gr
import pandas as pd
from datetime import datetime
from google.colab import drive
import matplotlib.pyplot as plt
from tabulate import tabulate
from IPython.display import display, HTML

import torch
import whisper
import librosa
import torchaudio
import soundfile as sf
import noisereduce as nr
from whisper import load_model
from pydub import AudioSegment
from pyAudioAnalysis import audioSegmentation as aS


from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


import tensorflow as tf
from keras.regularizers import l2
from keras.models import load_model
from keras.models import model_from_json
from tensorflow.keras import regularizers
from keras.layers import BatchNormalization
from tensorflow.keras.models import save_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Conv1D, MaxPooling1D, Flatten


from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, pipeline
from scipy.special import softmax

# Conexión al drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Cargar el modelo

In [ ]:
# Cargar el modelo neuronal y otros componentes
def load_neural_model_and_scaler():
    # Cargar la arquitectura del modelo
    with open('/content/drive/MyDrive/TrabajoFinal/Modelos/model_architecture.json', 'r') as json_file:
        model_json = json_file.read()
    model = model_from_json(model_json)

    # Cargar los pesos del modelo
    model.load_weights('/content/drive/MyDrive/TrabajoFinal/Modelos/best_model_neuronal_weights.h5')

    # Cargar los parámetros del modelo
    model_params = joblib.load('/content/drive/MyDrive/TrabajoFinal/Modelos/model_params.pkl')

    # Compilar el modelo con los parámetros cargados
    model.compile(optimizer=model_params['optimizer'], loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Cargar el scaler
    scaler = joblib.load('/content/drive/MyDrive/TrabajoFinal/Modelos/scaler.pkl')

    # Cargar el label encoder
    label_encoder = joblib.load('/content/drive/MyDrive/TrabajoFinal/Modelos/label_encoder.pkl')

    # Cargar las clases del codificador de etiquetas
    loaded_classes = np.load('/content/drive/MyDrive/TrabajoFinal/Modelos/classes.npy', allow_pickle=True)
    label_encoder.classes_ = loaded_classes

    return model, scaler, label_encoder

model, scaler, label_encoder = load_neural_model_and_scaler()


In [ ]:
# Cargar el modelo y arquitectura desde los archivos
def cargar_modelo_y_scaler(model_path, scaler_path):
    try:
        model = joblib.load(model_path)
        with open(scaler_path, 'r') as f:
            scaler_params = json.load(f)
        scaler = StandardScaler()
        scaler.mean_ = np.array(scaler_params['mean'])
        scaler.scale_ = np.array(scaler_params['scale'])
        return model, scaler
    except Exception as e:
        print(f"Error al cargar el modelo o scaler: {e}")
        return None, None


model_path = '/content/drive/MyDrive/TrabajoFinal/Modelos/model_weights.bin'
scaler_path = '/content/drive/MyDrive/TrabajoFinal/Modelos/model_architecture.json'

model, scaler = cargar_modelo_y_scaler(model_path, scaler_path)

# Conexión a la BD

In [ ]:
username = "USER_PROYFINAL"
dsn = "dbproyfinal_high"
pw = "UserProyFinal#40_24"
wallet_pw="^&$#7aBcdeFgHiJkLmNpQrStUvWxYz@!%"

In [ ]:
try:
    conn = oracledb.connect(
    user= username,
    password= pw,
    dsn= dsn,
    #Colocar ruta del walet en la maquina donde se ejecute, el wallet debe estar descomprimido
    config_dir=r'/content/drive/MyDrive/TrabajoFinal/Codigo/DB/Wallet_DBPROYFINAL',
    wallet_location=r'/content/drive/MyDrive/TrabajoFinal/Codigo/DB/Wallet_DBPROYFINAL',
    wallet_password=wallet_pw
    )
    # Print the version of the database to confirm connection
    print(conn.version)
except Exception as err:
    print("Error en la conexión a la base de datos", err)

cur_01=conn.cursor()


##Cierra la conexion de la base de datos
#conn.close()

def write_db(pfile_name, pupload_date, pfile_size, pclient, pcalldate, pparticipants,  pemployee, pai_value):
    ##Instrucciones para consultar datos de la tabla prueba2
    try:
        insert_datos = '''insert into analisis_sentimientos(file_name,
        upload_date,
        file_size,
        client,
        calldate,
        participants,
        employee,
        ai_value)
        values(:tagfile_name, :tagupload_date, :tagfile_size, :tagclient, :tagcalldate, :tagparticipants,  :tagemployee, :tagai_value)'''
        cur_01.execute(insert_datos,tagfile_name=pfile_name, tagupload_date=pupload_date, tagfile_size=pfile_size, tagclient=pclient, tagcalldate=pcalldate, tagparticipants=pparticipants,  tagemployee=pemployee,tagai_value=pai_value)
    except Exception as err:
        print("Error insertando datos",err)
    else:
        print("Datos insertados correctamente")
        conn.commit()

23.5.0.24.6


# Métodos para el análisis de emociones

In [ ]:
emociones={
        '0':'Angry',
        '1':'Disgust',
        '2':'Happy',
        '3':'Neutral',
        '4':'Sad',
        '5':'VeryAngry',
        '6':'VeryDisgust',
        '7':'VeryHappy',
        '8':'VerySad'
        }

# Función para escribir el texto del audio
def speech_to_text(audio, company, employee, calldate, participants):
  try:
    model = whisper.load_model("base")
    result = model.transcribe(audio)
    texto = result["text"]
    # Ruta destino del archivo en la carpeta deseada
    dest_path = os.path.join(os.path.basename(audio))
    shutil.move(audio, dest_path)
    AudiosinExtension = os.path.splitext(dest_path)[0]

    file = open(AudiosinExtension+".txt", "w")
    file.write(texto)
    file.close()

    AnalisisSentimientoTexto = analisis_sentimiento_texto(texto, company, employee, calldate, participants,origen="1")
    AnalisisSentimientoVoz = analisis_sentimiento_voz(dest_path)
    AnalisisConjunto = AnalisisSentimientoConjunto(AnalisisSentimientoTexto, AnalisisSentimientoVoz)

    return AnalisisConjunto

  except Exception as e:
    print(f"Error: {e}")
    return f"Se produjo un error en el análisis: {e}"

def analisis_sentimiento_texto(text, company, employee, calldate, participants,origen):

  # se separa el texto por lineas para hacer la separación
  lineas = text.split('\n')
  # Lista para almacenar las líneas del cliente
  lineas_cliente = []

  # Variable para indicar si estamos dentro de un diálogo del cliente
  es_dialogo = False

  # Itera sobre las líneas del archivo
  if origen == "0":
      for linea in lineas:
          if es_dialogo and linea != "":
              lineas_cliente.append(linea.strip())

          # Verifica si la línea actual es dicha por el cliente
          if linea.startswith(employee):
              es_dialogo = True
          elif linea == '':
              es_dialogo = False

  elif origen=="1":
      lineas_cliente=lineas[0].split('.')

  # Imprimir el contenido en bloques de 15 líneas
  bloque_size = 15
  total_lineas = len(lineas_cliente)
  valores = []
  for i in range(0, total_lineas, bloque_size):
    parrafo = ''
    bloque = lineas_cliente[i:i + bloque_size]
    for linea in bloque:
        parrafo = parrafo + ' ' + linea.strip()
    neg, neu, pos = analizar(parrafo)
    valores.append([neg, neu, pos])

  pneg, pneu, ppos = np.mean(valores, axis=0)
  satisfaccion = clasificar_satisfaccion(pneg, pneu, ppos)
  return satisfaccion

def analizar(texto):
  # creación del pipeline cargando el modelo preentrenado
  pipe = pipeline(
      model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
      top_k=None)
  # obtención de la predicción
  evaluacion = pipe(texto)[0]

  # procesamiento de la respuesta del modelo
  if evaluacion[0]['label']=='negative': neg = evaluacion[0]['score']
  elif evaluacion[0]['label']=='positive': pos = evaluacion[0]['score']
  else: neu = evaluacion[0]['score']

  if evaluacion[1]['label']=='negative': neg = evaluacion[1]['score']
  elif evaluacion[1]['label']=='positive': pos = evaluacion[1]['score']
  else: neu = evaluacion[1]['score']

  if evaluacion[2]['label']=='negative': neg = evaluacion[2]['score']
  elif evaluacion[2]['label']=='positive': pos = evaluacion[2]['score']
  else: neu = evaluacion[2]['score']

  return (neg, neu, pos)

# Definir los límites de cada categoría
def clasificar_satisfaccion(negativo, neutro, positivo):
    if positivo >= 0.8:
        return "Very satisfied"
    elif positivo >= 0.4 and positivo > negativo:
        return "Satisfied"
    elif negativo >= 0.8:
        return "Very unsatisfied"
    elif negativo >= 0.4 and negativo > positivo:
        return "Unsatisfied"
    elif neutro >= 0.4:
        return "Neutral"
    else:
        return "Neutral"

def obtener_emocion(numero_emocion):
    try:
        numero_emocion = str(numero_emocion)

        if numero_emocion in emociones:
            return emociones[numero_emocion]
        else:
            return "Emoción no encontrada"
    except Exception as e:
        return f"Error: {e}"

def analisis_sentimiento_voz(audio_path):
    audio_sin_extension = os.path.splitext(audio_path)[0]
    new_output_path = os.path.join("/content/drive/MyDrive/TrabajoFinal/AudiosCargados/", audio_sin_extension+".csv")
    output_dir_audio = "/content/drive/MyDrive/TrabajoFinal/AudiosCargados/"
    os.makedirs(output_dir_audio, exist_ok=True)
    data_new_audio = []
    valor = extract_features(audio_path, new_output_path)

    if valor == "Error al ejecutar OpenSMILE":
        return "Error al extraer características del nuevo archivo de audio"

    with open(new_output_path, 'r') as file:
        lines_new_audio = file.readlines()

    try:
        data_start_index = lines_new_audio.index('@data\n') + 1
    except ValueError:
        return "Error: '@data' no encontrado en el archivo."

    feature_names_new_audio = []
    feature_values_new_audio = []
    for line_new_audio in lines_new_audio:
        if line_new_audio.startswith('@attribute'):
            parts = line_new_audio.split()
            feature_names_new_audio.append(parts[1])
        elif not line_new_audio.startswith('@') and line_new_audio.strip():
            feature_values_new_audio = line_new_audio.strip().split(',')

    feature_data_new_audio = {feature_names_new_audio[i]: float(feature_values_new_audio[i]) if feature_values_new_audio[i] != '?' else None for i in range(1, len(feature_values_new_audio))}
    feature_data_new_audio['label'] = "emotion"
    data_new_audio.append(feature_data_new_audio)
    df_new_audio = pd.DataFrame(data_new_audio)
    df_new_audio.to_csv(new_output_path, index=False)

    new_audio_features = pd.read_csv(new_output_path, delimiter=',')
    new_audio_features = new_audio_features.drop(columns=['label', 'class'], errors='ignore')

    new_audio_features = scaler.transform(new_audio_features)

    if new_audio_features.ndim == 1:
        new_audio_features = new_audio_features.reshape(1, -1)

    prediction = model.predict(new_audio_features)
    predicted_label = obtener_emocion(np.argmax(prediction))

    nivel_satisfaccion_voz = ""
    if predicted_label in ("VeryAngry", "VerySad", "VeryDisgust"):
        nivel_satisfaccion_voz = "Very unsatisfied"
    elif predicted_label in ("Angry", "Sad", "Disgust"):
        nivel_satisfaccion_voz = "Unsatisfied"
    elif predicted_label == "Neutral":
        nivel_satisfaccion_voz = "Neutral"
    elif predicted_label == "Happy":
        nivel_satisfaccion_voz = "Satisfied"
    elif predicted_label == "VeryHappy":
        nivel_satisfaccion_voz = "Very satisfied"

    return nivel_satisfaccion_voz

def AnalisisSentimientoConjunto(SentimientoTexto, SentimientoAudio):
    # Definición de valores de sentimiento
    PesoSentimientos = {
        "Very satisfied": 1,
        "Satisfied": 0.75,
        "Neutral": 0.5,
        "Very unsatisfied": 0.25,
        "Unsatisfied": 0    }

    # Pesos
    PesoSentimientoTexto = 0.6
    PesoSentimientoAudio = 0.4

    # Valores numéricos de los sentimientos
    ValorTexto = PesoSentimientos[SentimientoTexto]
    ValorAudio = PesoSentimientos[SentimientoAudio]

    # Cálculo del promedio ponderado
    promedio_ponderado = (ValorTexto * PesoSentimientoTexto + ValorAudio * PesoSentimientoAudio) / (PesoSentimientoTexto + PesoSentimientoAudio)

    #Determinar el nivel de satisfacción correspondiente al promedio ponderado
    if promedio_ponderado >= 0.875:
        return "Very satisfied"
    elif promedio_ponderado >= 0.625:
        return "Satisfied"
    elif promedio_ponderado >= 0.375:
        return "Neutral"
    elif promedio_ponderado >= 0.125:
        return "Unsatisfied"
    else:
        return "Very unsatisfied"

# Interfaz

In [ ]:
styles = {
     "body": {
        "background-color": "white",
        "margin": "0",
        "padding": "0",
        "height": "100vh",
        "display": "flex",
        "justify-content": "center",
        "align-items": "center"
    },
    ".gradio-container": {
        "background-color": "white",
        "width": "100%",
        "height": "100%",
        "display": "flex",
        "justify-content": "center",
        "align-items": "center"
    },
    ".lg.primary.svelte-cmf5ev": {
        "background-color": "#5AAAF0",
        "color": "white",
        "font-size": "16px",
        "border-radius": "125px",
        "margin": "0px 10px"
    },
     ".lg.secondary.svelte-cmf5ev": {
        "background-color": "#B5B7CF",
        "color": "white",
        "font-weight": "bold",
        "font-size": "16px",
        "padding": "10px 20px",
        "border-radius": "125px",
        "margin": "20px 10px"
     },
    ".svelte-1bvc1p0 th": {
        "background-color": "#5AAAF0",
        "color": "white",
        "font-weight": "bold",
        "font-size": "14px",
        "border": "1px solid #ddd",
        "text-align": "center"
    },
    ".svelte-1bvc1p0 td": {
        "background-color": "white",
        "color": "black",
        "border": "1px solid #ddd",
        "text-align": "center"
    },
     ".block.svelte-12cmxck": {
        "background-color": "white",
        "color": "black",
        "text-align": "center",
        "border": "1px solid white"
    },
     ".svelte-1b6s6s": {
         "display": "inline-block",
         "background-color": "white",
         "color": "black",
         "border": "1px solid black"
    },
     ".svelte-j5bxrl": {
         "color": "#B5B7CF",
         "background-color": "#B5B7CF",
         "border": "1px solid black"
    },
     ".svelte-j5bxrl svg": {
         "color": "#B5B7CF",
         "margin-right": "10px"
    },
     ".wrap": {
        "color": "black"
    },
     ".or": {
         "color": "black"
    },
     ".svelte-1gfkn6j": {
         "color": "black"
    },
     ".svelte-sa48pu.stretch":{
         "background-color": "white",
         "border": "1px solid white",
         "align-items": "center",
         "justify-content": "center"
    },
     ".svelte-iyf88w": {
         "border": "1px solid white"
    },
     ".svelte-1f354aw textarea": {
         "border": "1px solid black",
         "background-color": "#B5B7CF",
         "font-weight": "bold",
         "color": "black"
    },
     ".svelte-vt1mxs": {
         "padding": "0"
    },
     "#component-231": {
         "margin-top": "10px"
    },
     ".svelte-vt1mxs gap": {
         "padding": "0"
     }
}


css_styles = "\n".join([f"{selector} {{{' '.join([f'{prop}: {value};' for prop, value in props.items()])}}}" for selector, props in styles.items()])

employees = ["Maylon Perez", "Richard Cloyed", "Brayden Nigro", "Donna Lacerda", "Chase Pollard", "Grant Brimdhall", "Lisa Dowing", "Ashley Chavez", "Gillian Chase"]

def on_upload_submit(file, company, employee, calldate, participants):
    try:
      participants = int(participants)
      if participants <= 0:
        return "Error: Se debe ingresar un número entero positivo en el campo participantes."
    except ValueError:
      return "Error: Se debe ingresar un número entero en el campo participantes."

    # Verificar que se haya subido un archivo
    if file:
        audio_sin_path = os.path.basename(file)
        audio_sin_extension = os.path.splitext(audio_sin_path)[0]
        path = '/content/drive/MyDrive/TrabajoFinal/AudiosCargados/'+audio_sin_extension+'.csv'

        if os.path.exists(path):
          os.remove(path)

        # Verificar si la fecha es válida
        try:
          datetime.strptime(calldate, '%Y-%m-%d')
        except ValueError:
          return "Error: La fecha proporcionada no tiene el formato correcto (YYYY-MM-DD) ó la fecha no existe."

        # Verificar si el archivo es un texto
        if ((file.name).find('.txt') != -1):
          with open(file.name, 'r') as f:
            content = f.read()

          # Verificar si el nombre del empleado en el contenido del archivo coincide
          if employee not in content:
            return "Error: Se ingresaron datos erróneos, por favor verificar que el nombre del empleado sea el mismo del que está en el transcript."

          valor = analisis_sentimiento_texto(content, company, employee, calldate, participants,origen="0")
          write_db(pfile_name=file.name, pupload_date=datetime.now().date(), pfile_size=100, pclient=company, pcalldate=calldate, pparticipants=int(participants), pemployee=employee, pai_value=valor)

          return valor

        # Verificar si el archivo es un audio
        elif ((file.name).find('.wav') != -1):
          audio_a_texto = speech_to_text(file, company, employee, calldate, participants)
          write_db(pfile_name=file.name, pupload_date=datetime.now().date(), pfile_size=100, pclient=company, pcalldate=calldate, pparticipants=int(participants), pemployee=employee, pai_value=audio_a_texto)

          file_input, company_input, employee_input, calldate_input, participants_input = clear_fields()

          return audio_a_texto
        else:
          return "El archivo cargado no tiene formato .txt o .wav."

    else:
        return "No se ha seleccionado ningún archivo."

def query_to_DB(param_client):
    try:
        select_datos = '''select upload_date,
        calldate,
        employee,
        client,
        participants,
        ai_value
        from analisis_sentimientos
        where client=:tagclient'''
        cur_01.execute(select_datos, tagclient=param_client)
        rows = cur_01.fetchall()
        for row in rows:
            print(row)
        return rows
    except Exception as err:
        print(f"Error:{err}")
        return f"Se produjo un error: {err}"


def clear_fields():
    return None,"", "", "", "",

with gr.Blocks(css=css_styles, theme=gr.themes.Monochrome()) as webpage:
    markdown_text = """
    <div style='text-align: center; font-size: 36px; color: Black; font-weight: bold'>
    Calls Sentiment Analysis
    </div>
    """
    gr.Markdown(markdown_text)

    # Crear botón cargar archivo, buscar y tabla de resultados
    new_text_file_button = gr.Button("Upload .txt or .wav file", variant="primary")
    new_query_button = gr.Button("Search call by Client", variant="primary")

    # Crear los componentes para el archivo de texto
    upload_file = gr.Group(visible=False)
    query_db = gr.Group(visible=False)

    with upload_file:
        with gr.Row():
            file_input = gr.File(label="Select a File", file_count="single", file_types=[".txt", ".wav" ], interactive=True)

            with gr.Row():
                company_input = gr.Textbox(label="Client")
                employee_input = gr.Dropdown(label="Employee", choices=employees)
            with gr.Row():
                calldate_input = gr.Textbox(label="Call date (YYYY-MM-DD)")
                participants_input = gr.Textbox(label="Participants")
            with gr.Row():
                close_button = gr.Button("Close", variant="secondary")
                upload_text_button = gr.Button("Upload", variant="primary")


            upload_text_button.click(on_upload_submit, inputs=[file_input, company_input, employee_input, calldate_input, participants_input], outputs=[gr.Textbox(label="Sentiment Analysis", lines=10)])
            upload_text_button.click(clear_fields, inputs=[], outputs=[file_input, company_input, employee_input, calldate_input, participants_input])
            close_button.click(lambda: gr.update(visible=False), inputs=[], outputs=[upload_file])


    with query_db:
        with gr.Row():
            cliente_query = gr.Textbox(label="Client")
            query_client_button = gr.Button("Search", variant="primary")
        lista_datos_db = gr.Dataframe(headers=["Date uploaded", "Call Date", "Employee", "Client", "Participants", "AI Value"],
                                          datatype=["date", "date", "str", "str", "number", "str"],
                                          row_count=1)
        query_client_button.click(query_to_DB, inputs=cliente_query, outputs=lista_datos_db)

    new_query_button.click(lambda: gr.update(visible=True), inputs=[], outputs=[query_db])
    new_text_file_button.click(lambda: gr.update(visible=True), inputs=[], outputs=[upload_file])

webpage.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://39f0e0e249b4918c45.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
